In [ ]:
import glob
import os
import pandas as pd
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [ ]:
paths = glob.glob("./raw_after_filtered/cicids2017/data/*.csv")
df = pd.DataFrame()

for path in paths:
    temp_df = pd.read_csv(path)
    df = pd.concat([df, temp_df])

df = df.dropna(how="any").dropna(axis=1, how="all")

ohe = OneHotEncoder(
    sparse=False,
    dtype=np.int8
)

df_ohe = ohe.fit_transform(df["Destination Port"])
df_ohe = pd.DataFrame(df_ohe, columns=ohe.get_feature_names_out(["Destination Port"]))

df_after = pd.concat([df, df_ohe], axis=1)
df_after = df_after.drop(columns=["Destination Port"])

In [ ]:
_, buf = train_test_split(
    df_after, test_size=0.02, random_state=42, stratify=df_after["Label"]
)

train, test = train_test_split(
    buf, test_size=0.2, random_state=42, stratify=buf["Label"]
)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

train_x = train.drop(columns=["Label"])
train_y = train["Label"]

smote_nc = SMOTENC(
    